In [29]:
import cv2
import pytesseract
import matplotlib.pyplot as plt
import os

In [30]:
def get_canny_edges(
    gray: cv2.Mat, blur_params=(5, 5), canny_params=(100, 200), canny_dilate=1
) -> cv2.Mat:
    """
    Get the Canny edges of an image.
    """
    gray = cv2.GaussianBlur(gray, blur_params, 0)
    edges = cv2.Canny(gray, canny_params[0], canny_params[1])
    edges = cv2.dilate(edges, None, iterations=canny_dilate)
    return edges

In [31]:
def extract_text_by_aspect_ratio(
    image: cv2.Mat, edges: cv2.Mat, gray: cv2.Mat
) -> list[str]:
    """
    Extrae texto de imágenes usando la relación de aspecto.
    Busca formas rectangulares que puedan contener texto.
    """
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    texts = []

    for contour in contours:
        area = cv2.contourArea(contour)
        x_pos, y_pos, width, height = cv2.boundingRect(contour)

        # Filtrar contornos muy pequeños
        if area < 800 or width < 50 or height < 15:
            continue

        # Aproximar la forma
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Asegurar que es una forma rectangular
        if len(approx) < 4 or len(approx) > 6:
            continue

        aspect_ratio = float(width) / height
        if not (1.5 <= aspect_ratio <= 5.0):  # Ajuste para matrículas comunes
            continue

        # Recortar la posible placa y extraer el texto
        placa = gray[y_pos : y_pos + height, x_pos : x_pos + width]
        text = pytesseract.image_to_string(placa, config="--psm 8").strip()

        # Filtrar textos no válidos
        if len(text) < 4 or not any(char.isdigit() for char in text):
            continue

        # Dibujar el rectángulo y mostrar el texto detectado
        cv2.rectangle(
            image, (x_pos, y_pos), (x_pos + width, y_pos + height), (0, 255, 0), 2
        )
        cv2.putText(
            image,
            text,
            (x_pos, y_pos - 5),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (0, 255, 0),
            2,
        )

        texts.append(text)

    return texts, image


In [32]:
def extract_text(edges, img, gray):
    return extract_text_by_aspect_ratio(img, edges, gray)

In [33]:
def get_slab_text(image_path: str, attempt=0, gray=None) -> tuple[list[str], cv2.Mat]:
    """
    Extrae texto de una imagen probando múltiples configuraciones.
    """
    attempts = [
        ((5, 5), (100, 200)),
        ((7, 7), (50, 150)),
        ((3, 3), (120, 250)),
        ((9, 9), (30, 100)),
        ((1, 1), (200, 300)),
        ((11, 11), (20, 80)),
        ((5, 5), (75, 150)),
        ((7, 7), (100, 200)),
        ((3, 3), (50, 100)),
    ]

    if attempt >= len(attempts):
        return [], None

    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: No se pudo cargar la imagen {image_path}")
        return [], None

    if gray is None:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    blur_params, canny_params = attempts[attempt]

    edges = get_canny_edges(gray, blur_params, canny_params)
    texts, updated_img = extract_text(edges, img, gray)

    if texts:
        return texts, updated_img

    return get_slab_text(image_path, attempt + 1, gray)

In [34]:
def get_all_files_in_directory(directory_path: str) -> list:
    """
    Get all files in a directory recursively.
    """
    if not os.path.exists(directory_path):
        print(f"Error: El directorio {directory_path} no existe.")
        return []

    return [
        os.path.join(directory_path, file)
        for file in os.listdir(directory_path)
        if file.lower().endswith(".png") or file.lower().endswith(".jpg")
    ]

In [35]:
directory_path = "./public/img/jpg/autos"
file_paths = get_all_files_in_directory(directory_path)

for file_path in file_paths:
    texts, updated_img = get_slab_text(file_path)
    print(f"File path: {file_path}, Text: {texts}")

File path: ./public/img/jpg/autos/auto.jpg, Text: ['CVL 657 18:']
File path: ./public/img/jpg/autos/ej01-auto.jpg, Text: ['.- 5']
File path: ./public/img/jpg/autos/ej02-auto.jpg, Text: ['7», =a']
File path: ./public/img/jpg/autos/ej03-auto.jpg, Text: []
File path: ./public/img/jpg/autos/ej04-auto.jpg, Text: ['p..BO0CDS']
